In [1]:
import sys
sys.path.append("../../")
# print(sys.path)

from IJCAI19.test.DefenseTest import *
# InitGlobal()

from attack import *
tf.flags.DEFINE_string(
        'dev_dir', '', 'dev directory with images.')
FLAGS.dev_dir = "../../../official_data/dev_data/"

FLAGS.input_dir = "../../../test_data/NonTargetAttackResult/"
FLAGS.output_file = "../../../test_data/DefenseResult.csv"
FLAGS.weight_path = "../../IJCAI19/weight/"
FLAGS.batch_size = 8

ModelFactory.WEIGHT_DIR = FLAGS.weight_path
TARGET_ATTACK = False

from IJCAI19.model.RandomDefense import RandomDefense
 

Using TensorFlow backend.


In [2]:
global_use_cpu()
FLAGS.batch_size = 32

In [5]:
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
name = "inception_v1"
T1 = RandomDefense(FLAGS.num_classes, name=name)
T1.random(
        msb_max=16, msb_rate=1, 
        rotation_range=20,
        width_shift_range=0.05,
        height_shift_range=0.05,
#         shear_range=0.05
)
name = "resnetv1_50"
T2 = RandomDefense(FLAGS.num_classes, name=name)
T2.random(
        msb_max=16, msb_rate=1, 
        rotation_range=20,
        width_shift_range=0.05,
        height_shift_range=0.05,
#         shear_range=0.05
)
name = "vgg_16"
T3 = RandomDefense(FLAGS.num_classes, name=name)
T3.random(
        msb_max=16, msb_rate=1, 
        rotation_range=20,
        width_shift_range=0.05,
        height_shift_range=0.05,
#         shear_range=0.05
)

name = "keras_xception_19"
T11 = RandomDefense(FLAGS.num_classes, name=name)
name = "keras_nasnet_large"
T12 = RandomDefense(FLAGS.num_classes, name=name)

D = EmbeddedDefenseModel("")
D.add_model(T11, weight=1)
D.add_model(T12, weight=1)
#     D.add_model(T2, weight=1)
#     D.add_model(T3, weight=1)

defense(D, repeat=1)

Validate()


without random_transform
loaded keras model from  ../../IJCAI19/weight/xception_19/keras_xception_19.h5
without random_transform
loaded keras model from  ../../IJCAI19/weight/nasnet/keras_nasnet_large.h5
correct 77/110,  Score 27.81301366710879


27.81301366710879